# Organização inicial

## Importações

In [1]:
from prepare_env import PrepareEnv
from configs import Configs
import os
import yfinance as yf
import pandas as pd
import datetime

In [2]:
preparer = PrepareEnv()
preparer.prepare_lib("yfinance")

yfinance já está instalado!


## Configurações iniciais

In [3]:
configs = Configs(os.getcwd(), '2025-01-01', '2025-02-17', True, False)

In [4]:
# Configuração temporária para exibir tudo
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

### Métodos principais

In [5]:
# Method para quebrar a lista de informações cadastrais
parametros = ['shortName', 'longName', 'longBusinessSummary']


def organizar_dados_cadastrais(ativo, retorno_api_cadastro, parametros):
    return [ativo][:6] + [retorno_api_cadastro.get(parametro) for parametro in parametros]

In [6]:
# Realiza a captura de dados dos ativos no YFinance
def capturar_dados_do_ativo(ativo):
    try:
        ticker = yf.Ticker(ativo)
        data = ticker.history(period="1d", interval='1m')
        info = ticker.info
        if data.empty:
            return None

        return organizar_dados_cadastrais(ativo, info, parametros)
    except:
        return None

In [7]:
# Gerar uma lista geral a partir da lista de ativos

def gerar_lista_geral(lista_de_ativos):
    lista_geral = []
    for ativo in lista_de_ativos:

        retorno = capturar_dados_do_ativo(ativo)
        if retorno is not None:
            lista_geral.append(retorno)

    return lista_geral

In [8]:
# Obter historico das cotacoes do ativo

def obter_historico_cotacoes(ativo, data_inicial, data_final):
    intervalo = '1h'
    cotacoes = yf.download(ativo, start=data_inicial, end=data_final, interval=intervalo)
    cotacoes['Ativo'] = ativo[:6]
    if isinstance(cotacoes.columns, pd.MultiIndex):
        cotacoes.columns = cotacoes.columns.droplevel(1)
    cotacoes.rename(
        columns={'Price Datetime': 'Data do Fechamento', 'Adj Close': 'Fechamento Ajustado', 'Close': 'Fechamento',
                 'High': 'Maior Cotação', 'Low': 'Menor Cotação', 'Volume': 'Volume'}, inplace=True)
    cotacoes.reset_index(inplace=True)
    return cotacoes

In [9]:
# Acumular dados de lista de ativos
def compilar_cotacoes_dos_ativos(lista_de_ativos, data_inicial, data_final):
    lista_geral = pd.DataFrame()
    ativos_sem_resultado = []

    for ativo in lista_de_ativos:
        try:
            cotacoes_do_ativo = obter_historico_cotacoes(ativo, data_inicial, data_final)
            if cotacoes_do_ativo is None or len(cotacoes_do_ativo) == 0:
                print(f"Não foi possível obter dados para o ativo {ativo}")
                ativos_sem_resultado.append(ativo)
                continue
            else:
                lista_geral = pd.concat([lista_geral, cotacoes_do_ativo],
                                        ignore_index=True)  # ignore_index para evitar duplicação de índices

        except Exception as e:
            print(f"Ocorreu um erro ao processar o ativo {ativo}: {e}")
            continue

    print(">>>>>>>>>>>>>>>>>>>>>>>>>>>>>>> Ativos sem resultado:", ativos_sem_resultado)

    return lista_geral

In [10]:
# Retornar uma lista com os tickers para iteração
def retornar_lista_de_tickers(lista_de_ativos, coluna):
    if coluna in lista_de_ativos.columns:

        tickers = lista_de_ativos[coluna].tolist()
        tickers_ajustado = [novo_ticker + '11.SA' for novo_ticker in tickers]

        return tickers_ajustado

    else:
        print(f"A coluna '{coluna}' não existe no DataFrame.")

In [11]:
# Obter historico das cotacoes do ativo

def obter_historico_de_dividendos(ativo, data_inicial, data_final):
    ticker = yf.Ticker(ativo)

    # Obtendo os dividendos
    info = ticker.dividends
    info = info[(info.index >= data_inicial) & (info.index <= data_final)]

    # Renomeando a coluna para 'Dividendos'
    info = info.rename('Dividendos').reset_index()
    info['Ativo'] = ativo[:6]  # Adicionando o ativo como nova coluna

    return info

In [12]:
# Acumular dados de lista de ativos
def compilar_dividendos_dos_ativos(lista_de_ativos, data_inicial, data_final):
    lista_geral = pd.DataFrame()
    ativos_sem_resultado = []

    for ativo in lista_de_ativos:
        try:
            cotacoes_do_ativo = obter_historico_de_dividendos(ativo, data_inicial, data_final)
            if cotacoes_do_ativo is None or len(cotacoes_do_ativo) == 0:
                print(f"Não foi possível obter dados de DIVIDENDOS para o ativo {ativo}")
                ativos_sem_resultado.append(ativo)
                continue
            else:
                lista_geral = pd.concat([lista_geral, cotacoes_do_ativo],
                                        ignore_index=True)  # ignore_index para evitar duplicação de índices

        except Exception as e:
            print(f"Ocorreu um erro ao processar os DIVIDENDOS do ativo {ativo}: {e}")
            continue

    print(">>>>>>>>>>>>>>>>>>>>>>>>>>>>>>> Ativos sem resultado:", ativos_sem_resultado)

    return lista_geral

# Captura de dados

## Lista de todos os fundos imobiliários

In [13]:
# Faz a leitura de todos os fundos imobiliários disponíveis na B3
fundos_imobiliarios = pd.read_csv(configs.arquivo_com_lista_de_fundos_imobiliarios, encoding='ISO-8859-1',
                                  delimiter=';')
fundos_imobiliarios.drop(columns=['Fundo', 'Segmento'], inplace=True)
fundos_imobiliarios.head()

,Razão Social,Código
0,AF INVEST CRI FDO. INV. IMOB - RECEBÍVEIS IMOB.,AFHI
1,AJ MALLS FDO DE INV IMOB,AJFI
2,ALIANZA CRÉDITO IMOBILIÁRIO FUNDO DE INVESTIM...,ALZC
3,ALIANZA MULTIESTRATÉGIA FUNDO DE INVESTIMENTO ...,ALZM
4,ALIANZA MULTIOFFICES - FDO. INV. IMOB,MTOF


In [14]:
# Gerar lista com os fundos imobiliários
lista_de_fundos_imobiliarios = retornar_lista_de_tickers(fundos_imobiliarios, 'Código')
print(lista_de_fundos_imobiliarios)

['AFHI11.SA', 'AJFI11.SA', 'ALZC11.SA', 'ALZM11.SA', 'MTOF11.SA', 'BICR11.SA', 'ALZR11.SA', 'AURB11.SA', 'APXR11.SA', 'APXU11.SA', 'AROA11.SA', 'ARXD11.SA', 'AIEC11.SA', 'AZPE11.SA', 'AZPL11.SA', 'BCRI11.SA', 'BNFS11.SA', 'BTML11.SA', 'BZEL11.SA', 'BPDR11.SA', 'BPLC11.SA', 'BBFI11.SA', 'BBFO11.SA', 'BBIG11.SA', 'BBIM11.SA', 'BBRC11.SA', 'RNDP11.SA', 'BGRB11.SA', 'BLOG11.SA', 'BLMG11.SA', 'BRSE11.SA', 'BCIA11.SA', 'CARE11.SA', 'RTEL11.SA', 'FATN11.SA', 'BRCD11.SA', 'BRCO11.SA', 'BICE11.SA', 'BIME11.SA', 'BRIM11.SA', 'BRIP11.SA', 'BIPD11.SA', 'BIPE11.SA', 'BRIX11.SA', 'BROF11.SA', 'BETW11.SA', 'LLAO11.SA', 'BTHI11.SA', 'BTLG11.SA', 'BTHF11.SA', 'BTYU11.SA', 'BTWR11.SA', 'BTSG11.SA', 'BTSI11.SA', 'CRFF11.SA', 'CXRI11.SA', 'CCME11.SA', 'CCVA11.SA', 'EVBI11.SA', 'CPLG11.SA', 'CPOF11.SA', 'CPFF11.SA', 'CPTS11.SA', 'CPSH11.SA', 'CACR11.SA', 'CBOP11.SA', 'BLCA11.SA', 'CENU11.SA', 'CFHI11.SA', 'CFII11.SA', 'CJCT11.SA', 'CIXM11.SA', 'CIXR11.SA', 'CLIN11.SA', 'CSMC11.SA', 'RENV11.SA', 'CVPR11.SA'

## Captura dos dados dos fundos imobiliários

### Arquivo com dados de cotações de fundos

In [15]:
# Consolidar o mapa de cotações
if configs.processar_cotacoes == True:
    todas_as_cotacoes = compilar_cotacoes_dos_ativos(lista_de_fundos_imobiliarios, configs.data_inicial,
                                                     configs.data_final)
    todas_as_cotacoes.to_csv(configs.arquivo_compilado_cotacoes, index=False)
    todas_as_cotacoes.head()

YF.download() has changed argument auto_adjust default to True


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed

1 Failed download:
['ALZM11.SA']: YFPricesMissingError('possibly delisted; no price data found  (1h 2025-01-01 -> 2025-02-17)')


Não foi possível obter dados para o ativo ALZM11.SA


[*********************100%***********************]  1 of 1 completed

1 Failed download:
['MTOF11.SA']: YFTzMissingError('possibly delisted; no timezone found')


Não foi possível obter dados para o ativo MTOF11.SA


...

Não foi possível obter dados para o ativo VDSV11.SA


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

Não foi possível obter dados para o ativo XPCE11.SA


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>> Ativos sem resultado: ['ALZM11.SA', 'MTOF11.SA', 'BICR11.SA', 'AURB11.SA', 'APXU11.SA', 'AZPE11.SA', 'BZEL11.SA', 'BPDR11.SA', 'BPLC11.SA', 'BBIM11.SA', 'BRSE11.SA', 'RTEL11.SA', 'BRCD11.SA', 'BIPE11.SA', 'BRIX11.SA', 'BETW11.SA', 'BTWR11.SA', 'BTSG11.SA', 'BTSI11.SA', 'CCVA11.SA', 'EVBI11.SA', 'CIXM11.SA', 'CIXR11.SA', 'CSMC11.SA', 'CVPR11.SA', 'CYLD11.SA', 'DLMT11.SA', 'EDGE11.SA', 'ERCR11.SA', 'FAGL11.SA', 'FARU11.SA', 'BMII11.SA', 'FIIC11.SA', 'IDGR11.SA', 'IDUA11.SA', 'PATB11.SA', 'BRHT11.SA', 'BVAR11.SA', 'CJFI11.SA', 'EZTB11.SA', 'HERM11.SA', 'FINF11.SA', 'JKDF11.SA', 'JKEF11.SA', 'MVFI11.SA', 'ESTQ11.SA', 'VPSI11.SA', 'QNTS11.SA', 'ANGC11.SA', 'SFFI11.SA', 'TGRU11.SA', 'TMFI11.SA', 'VERE11.SA', 'FBTS11.SA', 'IDFI11.SA', 'FAVM11.SA', 'FIGV11.SA', 'VTPL11.SA', 'ATWN11.SA', 'RECX11.SA', 'VTPA11.SA', 'VTXI11.SA', 'FTRE11.SA', 'BCFF11.SA', 'HRES11.SA', 'N4V111.SA', 'VCHG11.SA', 'FLNR11.SA', 'HLMB11.SA', 'SBCL11.SA', 'FRHY11.SA', 'SPVJ11.SA', 'HOMS11.S

## Arquivo com o resultado das cotações


### Execução

In [16]:
# Consolidar todo o mapa de cotações
if configs.processar_dividendos == True:
    todas_os_dividendos = compilar_dividendos_dos_ativos(lista_de_fundos_imobiliarios, configs.data_inicial,
                                                         configs.data_final)
    todas_os_dividendos.to_csv(configs.arquivo_compilado_dividendos, index=False)
    todas_os_dividendos.head()
    # prompt: rsult = obter_historico_de_dividendos ('BRCO11.SA')

    result = obter_historico_de_dividendos('BRCO11.SA', configs.data_inicial, configs.data_final)
    result

TypeError: Invalid comparison between dtype=datetime64[ns, America/Sao_Paulo] and date

# Pontos a evoluir no código